In [ ]:
## Project 4 - Interactive Crime Map of Baltimore
## ------------------------------------------------------------------------------------------------------------
## An interactive map of crime in Baltimore displaying all felonies commited in 2012. 
## Each point on the map shall be an individual felony in which the suspect was arrested. Each felony shall
## be assigned a different color in order to distinguish different types of felony. Clicking on any individual
## point shall show the arrest location, time, date, age, and sex of the suspect


In [1]:
import folium
import requests
import pandas
import numpy as np
import math

arrest_table = pandas.read_csv("http://www.hcbravo.org/IntroDataSci/misc/BPD_Arrests.csv")

arrest_table = arrest_table[pandas.notnull(arrest_table["Location 1"])]

arrest_table["lat"], arrest_table["long"] = arrest_table["Location 1"].str.split(",").str
arrest_table["lat"] = arrest_table["lat"].str.replace("(", "").astype(float)
arrest_table["long"] = arrest_table["long"].str.replace(")", "").astype(float)
arrest_table

arrest  age race sex  arrestDate arrestTime      arrestLocation  \
1       11127013.0   37    B   M  01/01/2011   00:01:00    2000 Wilkens Ave   
2       11126887.0   46    B   M  01/01/2011   00:01:00   2800 Mayfield Ave   
3       11126873.0   50    B   M  01/01/2011   00:04:00   2100 Ashburton St   
4       11126968.0   33    B   M  01/01/2011   00:05:00     4000 Wilsby Ave   
5       11127041.0   41    B   M  01/01/2011   00:05:00    2900 Spellman Rd   
...            ...  ...  ...  ..         ...        ...                 ...   
104522  13610388.0   32    B   M  12/31/2012   23:00:00     3300 Pulaski St   
104523  13610389.0   27    B   M  12/31/2012   23:10:00  400 W Baltimore St   
104524  13610377.0   34    B   M  12/31/2012   23:30:00      3800 Belair Rd   
104525  13610383.0   38    B   M  12/31/2012   23:40:00     2000 Mckean Ave   
104527  13610460.0   43    B   F  12/31/2012   23:55:00    4200 Towanda Ave   

              incidentOffense          incidentLocation  charge  \
1                    79-Other  Wilkens Av & S Payson St  1 1425   
2             Unknown Offense                       NaN     NaN   
3                    79-Other         2100 Ashburton St  1 1106   
4             Unknown Offense         1700 Aliceanna St     NaN   
5       81-Recovered Property           2900 Spelman Rd  1 1425   
...                       ...                       ...     ...   
104522               79-Other           3300 Pulaski Hw  1 5212   
104523        Unknown Offense                       NaN  2 0055   
104524      4E-Common Assault            3600 Belair Rd  1 1415   
104525           87-Narcotics            1700 Mckean Av  2A0696   
104527   4C-Agg. Asslt.- Oth.           4200 Towanda Av  1 1415   

                                  chargeDescription      district   post  \
1       Reckless Endangerment || Hand Gun Violation      SOUTHERN  934.0   
2                                    Unknown Charge  NORTHEASTERN  415.0   
3             Reg Firearm:Illegal Possession || Hgv       WESTERN  735.0   
4                                    Unknown Charge      NORTHERN  525.0   
5        Reckless Endangerment || Handgun Violation      SOUTHERN  924.0   
...                                             ...           ...    ...   
104522       Handgun On Person || Handgun Violation  SOUTHEASTERN  223.0   
104523     Fail Obey Renble/Lawfl || Disorderly/Fto       CENTRAL  113.0   
104524                  Asslt-Sec Degree || Assault  NORTHEASTERN  415.0   
104525  Att-Cds Manuf/Dist-Narc || Distribution Cds       WESTERN  733.0   
104527       Asslt-Sec Degree || 1St Degree Assault  NORTHWESTERN  613.0   

                  neighborhood                       Location 1        lat  \
1             Carrollton Ridge  (39.2814026274, -76.6483635135)  39.281403   
2                Belair-Edison  (39.3227699160, -76.5735750473)  39.322770   
3       Panway/Braddish Avenue  (39.3117196723, -76.6623546313)  39.311720   
4                     Pen Lucy  (39.3382885254, -76.6045667070)  39.338289   
5                  Cherry Hill  (39.2449886230, -76.6273582432)  39.244989   
...                        ...                              ...        ...   
104522   Ellwood Park/Monument  (39.2958396988, -76.5712467336)  39.295840   
104523                Downtown  (39.2893323126, -76.6210021717)  39.289332   
104524           Belair-Edison  (39.3252613570, -76.5689030474)  39.325261   
104525               Mondawmin  (39.3116837460, -76.6475011849)  39.311684   
104527        Towanda-Grantley  (39.3361054128, -76.6671512244)  39.336105   

             long  
1      -76.648364  
2      -76.573575  
3      -76.662355  
4      -76.604567  
5      -76.627358  
...           ...  
104522 -76.571247  
104523 -76.621002  
104524 -76.568903  
104525 -76.647501  
104527 -76.667151  

[63892 rows x 17 columns]

In [2]:
## Adding a new row for the type of charge based off of only felonies.
## Felonies include: 1st Degree Assault, Sexual Assault, Fraud, Murder, Robbery, Burglary, and Arson

charges = arrest_table['chargeDescription']
arrest_table['type'] = np.zeros(len(arrest_table))
ind = arrest_table.index

assault = []
sex = []
fraud = []
murder = []
robbery = []
burglary = []
arson = []
for i in ind:
    temp = str(charges[i])

    ## First degree Assault
    if "Asslt-First Degree" in temp:
        assault.append(i)
    elif "1St Degree Assault" in temp:
        assault.append(i)
    elif "Assault 1St Degree" in temp:
        assault.append(i)
        
    ## Sexual Assault/ Sexual offense/ Child Sexual Abuse
    elif "Sex" in temp:
        sex.append(i)
    elif "Fraud" in temp:
        fraud.append(i)
    elif "Murder" in temp:
        murder.append(i)
    elif "Robbery" in temp:
        robbery.append(i)
    elif "Burglary" in temp:
        burglary.append(i)
    elif "Arson" in temp:
        arson.append(i)
        

In [3]:
## Assign a number to the type based off of the type of felony

arrest_table['type'] = np.zeros(len(arrest_table))

for i in assault:
    arrest_table.at[i,'type'] = 1
for i in sex:
    arrest_table.at[i,'type'] = 2
for i in fraud:
    arrest_table.at[i,'type'] = 3
for i in murder:
    arrest_table.at[i,'type'] = 4
for i in robbery:
    arrest_table.at[i,'type'] = 5
for i in burglary:
    arrest_table.at[i,'type'] = 6
for i in arson:
    arrest_table.at[i,'type'] = 7
    
## Remove from the table all non-felony offences
arrest_table = arrest_table[arrest_table.type != 0]

## Remove from the table all felonies not done in 2012
date = arrest_table['arrestDate']
ind = arrest_table.index
arrest_table['year'] = np.zeros(len(arrest_table))
for i in ind:
    asfloat = float(date[i][8:])
    if (asfloat == 11):
        arrest_table.at[i,'year'] = 11
    elif (asfloat == 12):
        arrest_table.at[i,'year'] = 12
arrest_table = arrest_table[arrest_table.year != 11]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [15]:
## Assign each felony a color in order to destinguish them from one another.
# assault = 1 = red
# sex = 2 = pink
# fraud = 3 = green
# murder = 4 = black
# robbery = 5 = lightgray
# burglary = 6 = gray
# arson = 7 = darkred
# location, time, date, age, and sex

map_osm = folium.Map(location=[39.29, -76.61], zoom_start=11)

for i in range(1,len(arrest_table)):
    loc = arrest_table.iloc[i]
    pop = "Location: " + str(loc['incidentLocation']) + "\nDate: " + str(loc['arrestDate']) + "\nTime: " + str(loc["arrestTime"]) + "\nAge: " + str(loc['age']) + "\nGender: " + str(loc['sex'])
    color_ind = 'black'
    if (arrest_table.iloc[i]['type'] == 1):
        color_ind = 'red'
    elif (arrest_table.iloc[i]['type'] == 2):
        color_ind = 'pink'
    elif (arrest_table.iloc[i]['type'] == 3):
        color_ind = 'green'
    elif (arrest_table.iloc[i]['type'] == 4):
        color_ind = 'black'
    elif (arrest_table.iloc[i]['type'] == 5):
        color_ind = 'lightgray'
    elif (arrest_table.iloc[i]['type'] == 6):
        color_ind = 'gray'
    elif (arrest_table.iloc[i]['type'] == 7):
        color_ind = 'darkred'
    folium.Marker([arrest_table.iloc[i]['lat'], arrest_table.iloc[i]['long']],
                    popup=pop,
                    icon=folium.Icon(color=color_ind, icon = 'circle', prefix = 'fa')).add_to(map_osm)
map_osm

In [5]:
## Reasoning:
## Due to the large nature of the crime database it was impossible to graph all points of data, therefore 
## It seemed necessary to somehow prune the data, so it was decided that felonies are the most serious
## and therefore more important to display. Even so, only felonies resulted in over 2000 entries which 
## still proved to be difficult for the map to display, so it was further pruned to only display crimes
## in 2012. This smaller set of entries made the data much less unwieldy and ultimately resulted in a
## much more manable map.

arrest_table

arrest  age race sex  arrestDate arrestTime        arrestLocation  \
52870   12395914.0   23    B   M  01/01/2012   00:10:00       5000 Preston St   
52892   12395891.0   60    B   M  01/01/2012   02:00:00    1200 Shellbanks Rd   
52906   12395959.0   25    B   M  01/01/2012   04:09:00      400 N Clinton St   
52943   12396078.0   35    B   F  01/01/2012   18:00:00        2800 Carver Rd   
52952   12396121.0   33    B   M  01/01/2012   21:30:00    4400 Frankford Ave   
...            ...  ...  ...  ..         ...        ...                   ...   
104391  12610025.0   40    B   M  12/30/2012   15:00:00  6700 Reisterstown Rd   
104401  12610081.0   53    W   F  12/30/2012   17:15:00      2400 Shirley Ave   
104461  12610201.0   22    B   M  12/31/2012   09:30:00     3000 Westwood Ave   
104466  12610235.0   21    B   M  12/31/2012   10:55:00      4700 Old York Rd   
104527  13610460.0   43    B   F  12/31/2012   23:55:00      4200 Towanda Ave   

                  incidentOffense       incidentLocation  charge  \
52870        3B-Robb Highway (Ua)  1000-Blk Armistead Wy  1 0521   
52892         4B-Agg. Asslt.- Cut     1200 Shellbanks Rd  1 1425   
52906        5B-Burg. Res. (Att.)       400 N Clinton St  2A3030   
52943           4E-Common Assault         2800 Carver Rd  1 6505   
52952     5C-Burg. Res. (Noforce)      4400 Frankford Av  2 3030   
...                           ...                    ...     ...   
104391    3CO-Robb Comm-Other Wpn   6700 Reisterstown Rd  1 0600   
104401  8AO-Arson Sin Res Str-Occ        2400 Shirley Av  1 6502   
104461               87-Narcotics   1700 Bloomingdale Rd  2 3030   
104466      5A-Burg. Res. (Force)       4700 Old York Rd  2C3010   
104527       4C-Agg. Asslt.- Oth.        4200 Towanda Av  1 1415   

                                        chargeDescription      district  \
52870          Theft Less Than $100.00 || Unarmed Robbery  NORTHEASTERN   
52892         Reckless Endangerment || 1St Degree Assault      SOUTHERN   
52906       Att-Burglary-Fourth Degree || Attempt B And E  SOUTHEASTERN   
52943                             Arson/Threat || Assault      SOUTHERN   
52952     Burglary-Fourth Degree || Burglary - 4Th Degree  NORTHEASTERN   
...                                                   ...           ...   
104391  Resist/Interfere With Arrest || Commercial Arm...  NORTHWESTERN   
104401      Malicious Burning/1St Deg || Arson 1St Degree      NORTHERN   
104461            Burglary-Fourth Degree || Cds Violation  SOUTHWESTERN   
104466        Con-Burglary/2Nd Degree/General || Burglary      NORTHERN   
104527             Asslt-Sec Degree || 1St Degree Assault  NORTHWESTERN   

         post                neighborhood                       Location 1  \
52870   431.0           Armistead Gardens  (39.3069129142, -76.5552677284)   
52892   922.0                 Cherry Hill  (39.2501297933, -76.6311094410)   
52906   223.0       Ellwood Park/Monument  (39.2965661400, -76.5706440900)   
52943   924.0                 Cherry Hill  (39.2472799811, -76.6260669179)   
52952   435.0                   Frankford  (39.3360523016, -76.5495518054)   
...       ...                         ...                              ...   
104391  631.0        Reisterstown Station  (39.3582506244, -76.7052824949)   
104401  531.0                 Greenspring  (39.3359059900, -76.6586466061)   
104461  811.0  Northwest Community Action  (39.3080332653, -76.6682045590)   
104466  524.0             Richnor Springs  (39.3466972399, -76.6076119157)   
104527  613.0            Towanda-Grantley  (39.3361054128, -76.6671512244)   

              lat       long  type  year  
52870   39.306913 -76.555268   5.0  12.0  
52892   39.250130 -76.631109   1.0  12.0  
52906   39.296566 -76.570644   6.0  12.0  
52943   39.247280 -76.626067   7.0  12.0  
52952   39.336052 -76.549552   6.0  12.0  
...           ...        ...   ...   ...  
104391  39.358251 -76.705282   5.0  12.0  
104401  39.335906 -76.658647   7.0  12.0